# LangChain Learning

## LangChain: Models, Prompts and Output Parsers

- OpenAI 的接口
- LangChain 的接口
  - model
  - prompt
  - output parsers

### 填写 OpenAI APi key

In [ ]:
#!pip install python-dotenv
#!pip install openai

In [2]:
import os
import openai

openai.api_key = os.environ['OPENAI_API_KEY']

## OpenAi 的 chat api

In [3]:
def get_completion(prompt, model="gpt-3.5-turbo"):
    messages = [{"role": "user", "content": prompt}]
    response = openai.ChatCompletion.create(
        model=model,
        messages=messages,
        temperature=0, 
    )
    return response.choices[0].message["content"]


In [4]:
get_completion("什么是 langchain")

'Langchain是一种基于区块链技术的语言交流平台，旨在为全球用户提供高效、安全、便捷的语言学习和交流服务。通过Langchain，用户可以选择自己感兴趣的语言进行学习，并与来自世界各地的语言学习者进行交流和互动。同时，Langchain还提供了一种去中心化的语言认证机制，可以帮助用户证明自己的语言水平，为未来的学习和职业发展打下基础。'

In [5]:
customer_conetent = """
Langchain是一种基于区块链技术的语言交流平台，\
旨在为全球用户提供高效、安全、便捷的语言学习和交流服务。\
通过Langchain，用户可以选择自己感兴趣的语言进行学习，并与来自世界各地的语言学习者进行交流和互动。\
同时，Langchain还提供了一种去中心化的语言认证机制，可以帮助用户证明自己的语言水平，为未来的学习和职业发展打下基础。
"""

In [15]:
style = "简短、口语化"

In [16]:
prompt = f"""
用指定的方式重新述说这段话，指定的方式是{style}
这段话：```{customer_conetent}```
"""
print(prompt)


用指定的方式重新述说这段话，指定的方式是简短、口语化
这段话：```
Langchain是一种基于区块链技术的语言交流平台，旨在为全球用户提供高效、安全、便捷的语言学习和交流服务。通过Langchain，用户可以选择自己感兴趣的语言进行学习，并与来自世界各地的语言学习者进行交流和互动。同时，Langchain还提供了一种去中心化的语言认证机制，可以帮助用户证明自己的语言水平，为未来的学习和职业发展打下基础。
```



In [17]:
response = get_completion(prompt)

In [18]:
print(response)

Langchain是一个用区块链技术做的语言交流平台，让你可以高效、安全、方便地学习和交流语言。你可以选择自己感兴趣的语言学习，和来自世界各地的语言学习者互动。还有一个去中心化的语言认证机制，可以帮助你证明自己的语言水平，为未来的学习和职业发展打下基础。


### LangChain 的 chat api

In [ ]:
#!pip install --upgrade langchain

#### Model

在LangChain中的模型主要分为三类
- LLMs: 基础大模型，可以直接使用openai的text-ada-001、text-davinci-003这些，也包含其他提供方的模型
- Chat Models: 对话模型
- Text Embedding Models: 文本向量模型

In [1]:
from langchain.chat_models import ChatOpenAI

In [3]:
chat = ChatOpenAI(temperature=0.0)
chat

ChatOpenAI(verbose=False, callbacks=None, callback_manager=None, client=<class 'openai.api_resources.chat_completion.ChatCompletion'>, model_name='gpt-3.5-turbo', temperature=0.0, model_kwargs={}, openai_api_key=None, openai_organization=None, request_timeout=None, max_retries=6, streaming=False, n=1, max_tokens=None)

#### Prompts

跟`Model`相似，`Prompt`也是分为了`Prompt Template`和`Chat Prompt Template`两种，主要原因是在chat模式下是有角色区分的。

##### Prompt Template

In [5]:
from langchain.prompts import PromptTemplate
from langchain.llms import OpenAI

template = """
我有一个{animals}宠物，应该起个什么名字？用中文回答:
"""
prompt_template = PromptTemplate(
    input_variables=['animals'],
    template=template
)
print(f"prompt_template:'{prompt_template}'\n\n'")

llm = OpenAI(model_name="text-davinci-003")

llm_result = llm.generate([prompt_template.format_prompt(animals='猫').text, prompt_template.format_prompt(animals='狗').text])
print(prompt_template.format_prompt(animals='猫').text, llm_result.generations[0][0].text,
      '\n\n\n',
      prompt_template.format_prompt(animals='狗').text, llm_result.generations[1][0].text)

prompt_template:'input_variables=['animals'] output_parser=None partial_variables={} template='\n我有一个{animals}宠物，应该起个什么名字？用中文回答:\n' template_format='f-string' validate_template=True'

'

我有一个猫宠物，应该起个什么名字？用中文回答:
 
可以取名"阿猫"、"悠悠"、"黑猫"、"小花"、"小黑"、"小白"等。 


 
我有一个狗宠物，应该起个什么名字？用中文回答:
 
我的宠物狗可以取名为“小黑”。


##### Chat Prompt Template

In [6]:
template_string = """
用指定的方式重新述说这段话，指定的方式是{style}
这段话：```{customer_conetent}```
"""

In [7]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

human_template = HumanMessagePromptTemplate.from_template(template_string)
prompt_template = ChatPromptTemplate.from_messages([human_template])

prompt_template

ChatPromptTemplate(input_variables=['customer_conetent', 'style'], output_parser=None, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['customer_conetent', 'style'], output_parser=None, partial_variables={}, template='\n用指定的方式重新述说这段话，指定的方式是{style}\n这段话：```{customer_conetent}```\n', template_format='f-string', validate_template=True), additional_kwargs={})])

In [8]:
prompt_template.messages[0].prompt.input_variables

['customer_conetent', 'style']

**`ChatPromptTemplate`没有`from_template`方法**

注意在现在的版本`ChatPromptTemplate`是没有`from_template`这个方法的，因为`ChatPromptTemplate`需要`from_messages`来引入`system`和`user`两个角色。


> ``` python
> ChatPromptTemplate.from_template(template_string)
> 报错如下：
> AttributeError                            Traceback (most recent call last)
> Cell In[13], line 1
> ----> 1 ChatPromptTemplate.from_template(template_string)
> AttributeError: type object 'ChatPromptTemplate' has no attribute 'from_template'
> ```


如果想用`from_template`方法可以替换成：

```python
from langchain.prompts import ChatMessagePromptTemplate
ChatMessagePromptTemplate.from_template(role="user", template=template_string)
```

In [11]:
customer_content = """
Langchain是一种基于区块链技术的语言交流平台，\
旨在为全球用户提供高效、安全、便捷的语言学习和交流服务。\
通过Langchain，用户可以选择自己感兴趣的语言进行学习，并与来自世界各地的语言学习者进行交流和互动。\
同时，Langchain还提供了一种去中心化的语言认证机制，可以帮助用户证明自己的语言水平，为未来的学习和职业发展打下基础。
"""
customer_style = "简短、口语化"

In [ ]:
from langchain.prompts import ChatMessagePromptTemplate

chat([ChatMessagePromptTemplate.from_template(role="user", template=template_string).format(customer_conetent=customer_content, style=customer_style)])

In [21]:
customer_messages = prompt_template.format_messages(customer_conetent=customer_content, style=customer_style)
customer_messages

[HumanMessage(content='\n用指定的方式重新述说这段话，指定的方式是简短、口语化\n这段话：```\nLangchain是一种基于区块链技术的语言交流平台，旨在为全球用户提供高效、安全、便捷的语言学习和交流服务。通过Langchain，用户可以选择自己感兴趣的语言进行学习，并与来自世界各地的语言学习者进行交流和互动。同时，Langchain还提供了一种去中心化的语言认证机制，可以帮助用户证明自己的语言水平，为未来的学习和职业发展打下基础。\n```\n', additional_kwargs={}, example=False)]

In [22]:
type(customer_messages)

list

In [23]:
type(customer_messages[0])

langchain.schema.HumanMessage

In [24]:
customer_response = chat(customer_messages)

In [25]:
customer_response.content

'Langchain是一个用区块链技术做的语言交流平台，让你可以高效、安全、方便地学习和交流语言。你可以选择自己感兴趣的语言学习，和来自世界各地的语言学习者互动。还有一个去中心化的语言认证机制，可以帮助你证明自己的语言水平，为未来的学习和职业发展打下基础。'

#### Output Parsers

In [26]:
{
  "function": "笔记",
  "create_time": '1991-01',
  "user_number": 100
}

{'function': '笔记', 'create_time': '1991-01', 'user_number': 100}

In [27]:
customer_review = """\
Notion是一款集成了笔记、知识库、数据表格、看板、日历等多种能力于一体的应用程序，它支持个人用户单独使用，也可以与他人进行跨平台协作。\
Notion由Ivan Zhao、Simon Last于2016年（也有说2013年，独立由Simon Last）在旧金山创立的。\
截至2021年10月，Notion估值103亿美元，在全球拥有超2000万用户，团队规模为180人左右。
"""

review_template = """\
根据文字提取下面的信息

function: 这个产品的用途。如果这个信息找不到就显示未知。

create_time: 这个产品的创建日期。如果这个信息找不到就显示-99。

user_number: 到目前为止这个产品有多少用户在使用。如果找不到这个信息就返回-1。

按照下面的keys将输出格式化为json:
function
create_time
user_number

text: {text}
"""

In [28]:
from langchain.prompts import ChatPromptTemplate, HumanMessagePromptTemplate

prompt_template = ChatPromptTemplate.from_messages([HumanMessagePromptTemplate.from_template(review_template)])

prompt_template

ChatPromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['text'], output_parser=None, partial_variables={}, template='根据文字提取下面的信息\n\nfunction: 这个产品的用途。如果这个信息找不到就显示未知。\n\ncreate_time: 这个产品的创建日期。如果这个信息找不到就显示-99。\n\nuser_number: 到目前为止这个产品有多少用户在使用。如果找不到这个信息就返回-1,=。\n\n按照下面的keys将输出格式化为json:\nfunction\ncreate_time\nuser_number\n\ntext: {text}\n', template_format='f-string', validate_template=True), additional_kwargs={})])

In [29]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

{
  "function": "集成了笔记、知识库、数据表格、看板、日历等多种能力于一体的应用程序，支持个人用户单独使用，也可以与他人进行跨平台协作。",
  "create_time": "2016年",
  "user_number": "超2000万"
}


In [30]:
type(response.content)

str

**通过`LangChain`的`Parsers`将字符串转换为python字典**

In [31]:
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

In [33]:
response_schemas = [
    ResponseSchema(name='function', description='这个产品的用途。如果这个信息找不到就显示未知'),
    ResponseSchema(name='create_time', description='这个产品的创建日期。如果这个信息找不到就显示-99'),
    ResponseSchema(name='user_number', description='到目前为止这个产品有多少用户在使用。如果找不到这个信息就返回-1'),
]
output_parsers = StructuredOutputParser(response_schemas=response_schemas)
output_parsers

StructuredOutputParser(response_schemas=[ResponseSchema(name='function', description='这个产品的用途。如果这个信息找不到就显示未知'), ResponseSchema(name='create_time', description='这个产品的创建日期。如果这个信息找不到就显示-99'), ResponseSchema(name='user_number', description='到目前为止这个产品有多少用户在使用。如果找不到这个信息就返回-1')])

In [35]:
format_instructions = output_parsers.get_format_instructions()
format_instructions

'The output should be a markdown code snippet formatted in the following schema, including the leading and trailing "\\`\\`\\`json" and "\\`\\`\\`":\n\n```json\n{\n\t"function": string  // 这个产品的用途。如果这个信息找不到就显示未知\n\t"create_time": string  // 这个产品的创建日期。如果这个信息找不到就显示-99\n\t"user_number": string  // 到目前为止这个产品有多少用户在使用。如果找不到这个信息就返回-1\n}\n```'

In [42]:
prompt_template = ChatPromptTemplate(
    messages=[HumanMessagePromptTemplate.from_template(review_template)], 
    input_variables=["text"],
    partial_variables={"format_instructions": format_instructions}
)

In [49]:
messages = prompt_template.format_messages(text=customer_review)
chat = ChatOpenAI(temperature=0.0)
response = chat(messages)
print(response.content)

{
  "function": "集成了笔记、知识库、数据表格、看板、日历等多种能力于一体的应用程序，支持个人用户单独使用，也可以与他人进行跨平台协作。",
  "create_time": "2016年",
  "user_number": "超2000万"
}


In [51]:
output_parsers.parse(response.content)

OutputParserException: Got invalid return object. Expected markdown code snippet with JSON object, but got:
{
  "function": "集成了笔记、知识库、数据表格、看板、日历等多种能力于一体的应用程序，支持个人用户单独使用，也可以与他人进行跨平台协作。",
  "create_time": "2016年",
  "user_number": "超2000万"
}